In [ ]:
data_source = r'D:\Data\AAPL.xlsx'
df1.to_excel(data_source)

In [ ]:
import pandas_datareader as pdr
df = pdr.get_data_yahoo('AAPL')
df1 = df[df['Volume']>0]
df1.tail(10)

In [ ]:
import pandas as pd
import numpy as np
datafile = r'D:\Data\AAPL.xlsx'
df = pd.read_excel(datafile)
df.tail(5)

In [ ]:
# our strategy is if the short term moving average of the adjusted close price is higher the long term moving average of the adjusted close price, we are going to generate a buy singal; if short term moving average is lower than the long-term moving average then we are going to sell, the signal will be negative one;
class Strategy:
    def _init_(self):
        self.t_name = t_name
        self.long_win = long_win
        self.short_win = short_win
        self.cond = df.index > self.long_win
        self.trade_price = df['Open']
        self.close = df['Adj  Close']
    
    def smav(self):
        self.smav = np.where(Strategy().cond,Strategy().close.rolling(window = Strategy.short_win).mean(),0)
        return self.smav
    
    
    def lmav(self):
        self.lmav = np.where(Strategy().cond,Strategy().close.rolling(window = Strategy.long_win).mean(),0)
        return self.lmav
    
    def trend_day(self):
        self.trend_day = np.where(Strategy().smav() > Strategy().lmav(),1,
                                 np.where(Strategy().smav() < Strategy().lmav(),-1,0))
        return self.trend_day
    
    def prev_trend_day(self):
        self.prev_trend_day = np.where(Strategy().self.cond, np.roll(Strategy().trend_day(),1),0)
        return self.prev_trend_day
    
    def diff_trend_day(self):
        self.diff_trend_day = Strategy().trend_day() + Strategy().prev_trend_day()
        return self.diff_trend_day
    
    
# Global Variables
t_name = 'mav' # moving average crossover
long_win = 20
short_win = 5
s = Strategy()

df['smav'] =  s.smav()
df['lmav'] =  s.lmav()
df['trend_day'] = s.trend_day()

df.tail(5)

In [ ]:
class Signal:
    def _init(self):
        pass
    def  trade_signal(self):
        self.trade_signal = np.where(Strategy().diff_trend_day() == 0,Strategy().trend_day(),0)
        return self.trade_signal
    def order(self):
        self.order = np.where(Strategy().cond, np.roll(Signal().trade_signal(),1),0)
        return self.order
    
ts = Signal()
df['trade_signal'] = ts.trade_signal()
df['order'] = ts.order()

df[85:125]



In [ ]:
class Portfolio:
    
    def _init_(self):
        self.lot_size_long = 1 
        self.lot_size_short = 1
        self.contract_size = np.where(period1,100,50)
        self.initial_cash = 100000
        self.long_amt = (-1)*(np.where(Signal().order()==1,self.lot_size_long*contract_size*Strategy().trade_price,0))
        self.short_amt = (1)*(np.where(Signal().order()==(-1),self.lot_size_short*contract_size*Strategy().trade_price,0))
    
    def cash_delta(self):
        self.cash_delta = Portfolio().long_amt + Porfolio().short_amt
        return self.cash_delta
    
    def end_bal(self):
        self.end_bal = Portfolio().initial_cash + Portfolio().cash_delta().cumsum()
        return self.end_bal
    
    def end_pos(self):
        self.end_pos = Signal().order().cumsum()
        return self.end_pos

p = Portfolio()

df['long_amt'] = p.long_amt
df['short_amt'] = p.short_amt
df['cash_delta'] = p.cash_delta()
df['end_bal'] = p.end_bal()
df['end_pos'] = p.end_pos()
df['pnl'] = df['end_bal'] + (Portfolio().end_pos()*Strategy().trade_price*Portfolio().contract_size) 
df.loc[100:200, ['Date','Open','smav','lmav','trade_signal','order','long_amt','short_amt','cash_delta','end_bal','end_pos']]

In [ ]:
# Plot gain and lost 
import matplotlib.pyplot as plt
%matplotlib inline

df1 = df.set_index('Date')
print_pnl = df1['pnl'].plot()
plt.show(print_pnl)